In [23]:
%%time
#!/usr/bin/env python
import sys
import gzip
import csv

#make sure that file exists
try:
    df = gzip.open("/projectnb/bf528/project_4_scrnaseq/fastq/SRR3879604/SRR3879604_1_bc.fastq.gz", "r")
except IndexError as ie:
    print("Call sort.py file_to_open file_to_write\n")

d ={} #dictionary for counts (barcode:count)
i=0 #for debugging fd
if df:
    for line in df:
        i+=1#fd
        if i > 100000000: #fd
            break #fd
        #if line has barcode, continue. Else skip
        elif line.startswith(b'@'): #after debugging, make this if
            bc = line.decode("utf-8").split(' ')
            #Only use lines w/barcodes
            if len(bc) > 2:
                #concat bc1 and bc2
                code = bc[1][4:] + bc[2][4:]
                #
                if code not in d:
                    d[code] = 1
                else:
                    d[code]+=1 
    
print(len(d))
print(i)
print(sum(d.values()))


285807
100000001
21696966
CPU times: user 2min 31s, sys: 155 ms, total: 2min 31s
Wall time: 2min 32s


In [32]:
print(sum(d.values()))

21651


In [48]:
#Test code for grouping barcodes

popped = [] #dictionary of popped keys
d2 = d.copy() #
final = {}
#compare all keys against each other
for key, value in d.items():
    p = []
    #If key in second dictionary, delete it from there and continue. Else skip it
    if key in d2:
        d2.pop(key)
        final[key]= value
        #compare key to the rest of the keys
        for key2, v2 in d2.items():
            tf = match2(key, key2)
            if tf <= 2 :
                popped.append([key2, key, tf]) #for traceback
                final[key] += v2
                p.append(key2) #remember that this value has already been used
    #set used values to 0
    for val in p:
        d2.pop(val)
#Inputs two strings, tells # of mismatches
#We allow deviations of 2 to be considered a match
def match2(key1, key2):
    score = 0
    if len(key1) != len(key2):
        return len(key1) + len(key2)
    for i in range(len(key1)):
            if key1[i] != key2[i]:
                score += 1
    return score
                               
print(sum(final.values()))
print(len(final))

21651
7556


In [47]:
#more test code for grouping bar codes
x = [popped[10][1]]
for l in popped:
    if l[1] == x[0]:
        x.append(l[0])
y = len(x)
for a in range(0,y):
    for b in range(a,y):
        c = match2(x[a],x[b])
        print(x[a],x[b],c)

AGCACCACACTGGAGTAAG AGCACCACACTGGAGTAAG 0
AGCACCACACTGGAGTAAG AGCACCACACTNGATAAAG 3
AGCACCACACTGGAGTAAG AGCACCACACTAAAGAAAG 3
AGCACCACACTGGAGTAAG AGCACCACACTGCACTCAG 3
AGCACCACACTGGAGTAAG TGCCTCACACTGGAGTAAG 3
AGCACCACACTNGATAAAG AGCACCACACTNGATAAAG 0
AGCACCACACTNGATAAAG AGCACCACACTAAAGAAAG 3
AGCACCACACTNGATAAAG AGCACCACACTGCACTCAG 5
AGCACCACACTNGATAAAG TGCCTCACACTGGAGTAAG 6
AGCACCACACTAAAGAAAG AGCACCACACTAAAGAAAG 0
AGCACCACACTAAAGAAAG AGCACCACACTGCACTCAG 5
AGCACCACACTAAAGAAAG TGCCTCACACTGGAGTAAG 6
AGCACCACACTGCACTCAG AGCACCACACTGCACTCAG 0
AGCACCACACTGCACTCAG TGCCTCACACTGGAGTAAG 6
TGCCTCACACTGGAGTAAG TGCCTCACACTGGAGTAAG 0


In [3]:
#Code to write to csv 
#very incomplete code
f = "dict_test.csv"
with open(f, "w") as file:
    w = csv.writer(file, delimiter=',')
    for key, value in d.items():
        w.writerow([key, value])

In [33]:
f = "/text.py"
if "/" in f:
    f = f.split("/")
    f = f[len(f)-1]
    f = ''.join(f)
f = f.split(".")
if len(f) > 1:
    f = f[:len(f)-1]
f = ''.join(f)
f += ".csv"

print(f)

text.csv


\
